Define our prompt

In [8]:

prompt = """
Generate a dataset of fake user orders. Each row of the dataset should be valid. The format should not be a list, it should be a JSON object.
${gr.complete_json_suffix}

an example of output may look like this:
{
	"user_orders": [{                                                                                                   │ │
        "user_id": 1,
        "user_name": "John Mcdonald",
    	"num_orders": 6
    }]
}

"""

Defined what our data should look like in pydantic

In [9]:
from pydantic import BaseModel, Field
from guardrails.validators import ValidLength, TwoWords, ValidRange
from typing import List

class Order(BaseModel):
	user_id: int = Field(description="The user's id.", validators=[("1-indexed", "noop")])
	user_name: str = Field(
		description="The user's first name and last name",
		validators=[TwoWords()]
	)

	num_orders: int = Field(
		description="The number of orders the user has placed",
		validators=[ValidRange(0, 50)]
	)
	


class Orders(BaseModel):
	user_orders: List[Order] = Field(
		description="Generate a list of users and how many orders they have placed in the past.",
		validators=[ValidLength(10, 10, on_fail="noop")]
	)

Set up the cohere client. In this notebook, we use an environment variable to pass the cohere API key through

In [10]:
import cohere
import os

api_key = os.environ['COHERE_API_KEY']

co = cohere.Client(api_key=api_key)

Create the guard and prompt

In [11]:
import guardrails as gd
guard = gd.Guard.from_pydantic(output_class=Orders, prompt=prompt)

raw_llm_response, validated_response, *rest = guard(
	co.generate,
	model="command",
	max_tokens=1024,
	temperature=0.3
)

By printing the guard's state, we can see the final, validated output, as well as the steps guardrails took to ensure structure.

In [13]:
from rich import print

print(guard.guard_state.most_recent_call.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Generate a dataset of fake user orders. Each row of the dataset should be valid. The format should not  │ │
    │ │ be a list, it should be a JSON object.                                                                  │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <list name="user_orders" format="length: min=10 max=10" description="Generate a list of users and   │ │
    │ │ how many orders they have placed in the past.">                                                         │ │
    │ │         <object>                                                                                        │ │
    │ │             <integer name="user_id" format="1-indexed" description="The user's id."/>                   │ │
    │ │             <string name="user_name" format="two-words" description="The user's first name and last     │ │
    │ │ name"/>                                                                                                 │ │
    │ │             <integer name="num_orders" format="valid-range: min=0 max=50" description="The number of    │ │
    │ │ orders the user has placed"/>                                                                           │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere,     │ │
    │ │ enter `null`.                                                                                           │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz':